In [ ]:
#1) Extract from S3: Create dataframe for table coding indstries
import pandas as pd
from smart_open import smart_open

df_industry = pd.read_csv(smart_open('s3://moz-sr-data-analyst/industries.csv'))

In [2]:
df_industry

,industry,industry_key
0,Marketing Agency,5
1,NaN,9
2,NaN,10
3,health care,1
4,healthcare,2
5,marketing agency,4
6,mktg agency,3
7,NaN,11
8,software,6
9,unk,8


In [3]:
#2) Extract from S3: Create dataframe for table containing column names for data
import pandas as pd
from smart_open import smart_open

df_columns = pd.read_csv(smart_open('s3://moz-sr-data-analyst/columns.tsv'))

In [4]:
df_columns

,column_name\tposition
0,id\t0
1,industry_key\t1
2,geography\t2
3,most_common_browser\t3
4,session_time_prior_month_seconds\t4
5,prior_month_paid\t5


In [ ]:
#3) Pull in a single parquet file to check columns

import s3fs

import pyarrow.parquet as pq

s3 = s3fs.S3FileSystem()

pandas_dataframe=pq.ParquetDataset('s3://moz-sr-data-analyst/website_logins/partition=1/c529fa6a3c1c4f59a479cfe09fef95b1.parquet',filesystem=s3).read_pandas().to_pandas()

pandas_dataframe.columns

In [41]:
#4) Slurp in all parquet 
import awswrangler as wr

df_data = wr.s3.read_parquet(path="s3://moz-sr-data-analyst/website_logins/")


In [42]:
df_data

,0,1,2,3,4,5
0,1418200170,1,US,IE,1682.0,105.932246
1,1418200182,4,Canada,Chrome,2824.0,117.570502
2,1418200189,4,Esp,IE,2236.0,77.349066
3,1418200194,2,Esp,Chrome,2480.0,48.719494
4,1418200206,5,United States,Chrome,285.0,113.691472
...,...,...,...,...,...,...
13014631,1431214776,3,United States,Safari,254.0,121.771472
13014632,1431214778,6,Pakistan,Firefox,1218.0,37.839101
13014633,1431214779,9,Pakistan,Chrome,6252.0,41.347359
13014634,1431214794,6,France,Safari,NaN,53.053718


In [8]:
#5) Assign column names to dataset
df_data.columns = ['id','industry_key','geography','most_common_browser','session_time_prior_month_seconds','prior_month_paid']


In [9]:
df_data

,id,industry_key,geography,most_common_browser,session_time_prior_month_seconds,prior_month_paid
0,1418200170,1,US,IE,1682.0,105.932246
1,1418200182,4,Canada,Chrome,2824.0,117.570502
2,1418200189,4,Esp,IE,2236.0,77.349066
3,1418200194,2,Esp,Chrome,2480.0,48.719494
4,1418200206,5,United States,Chrome,285.0,113.691472
...,...,...,...,...,...,...
13014631,1431214776,3,United States,Safari,254.0,121.771472
13014632,1431214778,6,Pakistan,Firefox,1218.0,37.839101
13014633,1431214779,9,Pakistan,Chrome,6252.0,41.347359
13014634,1431214794,6,France,Safari,NaN,53.053718


In [10]:
#6) Left outer join data (fact) and indsustry (dimesnsion) to append industry to dataset
df_test = pd.merge(df_data.assign(industry_key=df_data.industry_key.astype(str)), 
         df_industry.assign(industry_key=df_industry.industry_key.astype(str)), 
         how='left', on='industry_key')

In [30]:
df_test

,id,industry_key,geography,most_common_browser,session_time_prior_month_seconds,prior_month_paid,industry
0,1418200170,1,USA,IE,1682.0,105.932246,healthcare
1,1418200182,4,CAN,Chrome,2824.0,117.570502,mktg agency
2,1418200189,4,ESP,IE,2236.0,77.349066,mktg agency
3,1418200194,2,ESP,Chrome,2480.0,48.719494,healthcare
4,1418200206,5,USA,Chrome,285.0,113.691472,mktg agency
...,...,...,...,...,...,...,...
13014631,1431214776,3,USA,Safari,254.0,121.771472,mktg agency
13014632,1431214778,6,Pakistan,Firefox,1218.0,37.839101,software
13014633,1431214779,9,Pakistan,Chrome,6252.0,41.347359,NaN
13014634,1431214794,6,France,Safari,NaN,53.053718,software


In [35]:
#7) Standardize and simplify industry and geography variables. 

df_test['industry'] = df_test['industry'].replace(['marketing agency','Marketing Agency',],'mktg agency')
df_test['industry'] = df_test['industry'].replace(['health care',],'healthcare')
df_test['industry'] = df_test['industry'].replace(['unknown'],'unk')
df_test['geography'] = df_test['geography'].replace(['United States','US','America'],'USA')
df_test['geography'] = df_test['geography'].replace(['United Kingdom','GB','England'],'GBR')
df_test['geography'] = df_test['geography'].replace(['Canada','CA'],'CAN')
df_test['geography'] = df_test['geography'].replace(['Esp','Spain'],'ESP')
df_test['geography'] = df_test['geography'].replace(['Pakistan'],'PAK')
df_test['geography'] = df_test['geography'].replace(['India'],'IND')
df_test['geography'] = df_test['geography'].replace(['Greece'],'GRC')
df_test['geography'] = df_test['geography'].replace(['France'],'FRA')


In [36]:
df_test

,id,industry_key,geography,most_common_browser,session_time_prior_month_seconds,prior_month_paid,industry
0,1418200170,1,USA,IE,1682.0,105.932246,healthcare
1,1418200182,4,CAN,Chrome,2824.0,117.570502,mktg agency
2,1418200189,4,ESP,IE,2236.0,77.349066,mktg agency
3,1418200194,2,ESP,Chrome,2480.0,48.719494,healthcare
4,1418200206,5,USA,Chrome,285.0,113.691472,mktg agency
...,...,...,...,...,...,...,...
13014631,1431214776,3,USA,Safari,254.0,121.771472,mktg agency
13014632,1431214778,6,PAK,Firefox,1218.0,37.839101,software
13014633,1431214779,9,PAK,Chrome,6252.0,41.347359,NaN
13014634,1431214794,6,FRA,Safari,NaN,53.053718,software


In [37]:
#8) Check geography naming
df_test.geography.unique()

<StringArray>
['USA', 'CAN', 'ESP', 'FRA', 'UK', 'GRC', 'IND', 'PAK']
Length: 8, dtype: string

In [15]:
#9) Check browser naming
df_test.most_common_browser.unique()

<StringArray>
['IE', 'Chrome', 'Safari', 'Firefox']
Length: 4, dtype: string

In [38]:
#10) Check industry nameing
df_test.industry.unique()

array(['healthcare', 'mktg agency', 'unk', 'software', nan], dtype=object)

In [40]:
#11) Check summary statistics for variables in dataset
df_test.describe(include='all') 

,id,industry_key,geography,most_common_browser,session_time_prior_month_seconds,prior_month_paid,industry
count,1.301464e+07,13014636,13014636,13014636,1.262410e+07,1.301464e+07,11064245
unique,NaN,11,8,4,NaN,NaN,4
top,NaN,6,USA,Chrome,NaN,NaN,mktg agency
freq,NaN,1628560,5782588,6504727,NaN,NaN,4881955
mean,1.424707e+09,NaN,NaN,NaN,1.944942e+03,-2.416684e+02,NaN
std,3.757002e+06,NaN,NaN,NaN,2.616890e+03,3.261685e+03,NaN
min,1.418200e+09,NaN,NaN,NaN,1.000000e+00,-3.276800e+04,NaN
25%,1.421454e+09,NaN,NaN,NaN,6.090000e+02,6.234438e+01,NaN
50%,1.424707e+09,NaN,NaN,NaN,1.206000e+03,8.700910e+01,NaN
75%,1.427961e+09,NaN,NaN,NaN,2.289000e+03,1.107259e+02,NaN


In [43]:
#DATA VISUALIZATION QUESTIONS

#4)  What information seems to be informative about how much they paid the prior month?
#The population is bimodal, with  138K or 1% of customers all having negative spend of -32,768 accross all categrical variables which seems suspicious.
#I wonder what's going on and if something is wrong (mean and median are the same)?
https://public.tableau.com/profile/todd.stoltey#!/vizhome/Moz_16149003853700/PMP_Neg

#Histogram with customer distributions Prior Month Spend and Prior Month Session Time
https://public.tableau.com/profile/todd.stoltey#!/vizhome/Moz_16149003853700/Hist_Cust
    
#Average Prior Month Session Time also seems very consistent at about 32min accross geo region,industry, and most common browser type.
https://public.tableau.com/profile/todd.stoltey#!/vizhome/Moz_16149003853700/Heat_Pos_PMP

#North American customers in the Marketing industry on average pay the most in the prior month ($129) and represent 38% of the customer base.
https://public.tableau.com/profile/todd.stoltey#!/vizhome/Moz_16149003853700/PMP
https://public.tableau.com/profile/todd.stoltey#!/vizhome/Moz_16149003853700/Heat_Pos_PMP

#For customers owing Prior Month Spend, the bulk of them are in North America. When comparing across all variables in a heat map,
#unknown IE users in France owe the highest average prior month spend (-$377).

#Overall North American customers represent the majority of Moz's customer base too (66%).
https://public.tableau.com/profile/todd.stoltey#!/vizhome/Moz_16149003853700/PMP_GEO
    
#Common Browser Types appear to be similar across country, industry, and prior month paid (positive or negative).
    

#5) Which segment of customers should Moz target for marketing?
#Currently the majority of customers come from North America of which most are in the marketing industry. Moz should try reaching out to
#Europe and Asia to increase their user base.  For the North American user's, Moz should try marketing more to the healthcare and 
#software industries.

#25% customers have uknown or null industry values. Improving the identification of the customer's industry would better inform the 
#business.
https://public.tableau.com/profile/todd.stoltey#!/vizhome/Moz_16149003853700/PMP
    
#6) What would your recommened next steps for the business be?
#Continue doing the same thing for the North American customers. Try to increase the European and Asian customer bases. Also try to 
#focus on bringing in more customers in the Software and Healthcare industries.  Are there other industries besides Marketing,
#Software, and Healthcare to try to market too? 

#Fix the null and unkown Industry data.

#7) As an anaylyst, is there additional data you would recommend collecting about these users?
#What about finding customers in other industries besides marketing, software, and healthcare? Since search results are specific to local
#and most people use google as yellow pages, what about Tree Service, Auto Repair, Restaurant, Legal, Auto Sales/Repair, Real Estate, Home Repair,
#Roofing...
#Size of business would be informative. For instance Moz may want to target new customer owners of smaller local companies in the above categories?


#Bonus Question (Didn't do but have an idea)
#I might try a regression model. There appears to be a positive correlation between Prior Month Paid and Prior Month Session Time.
#Prior Month Paid increased with Prior Month Session Time. Looking a scatter density plot (Prior Month Paid Vs Prior Month Session Time), it looks like Prior Month Paid is clustered 
#into six groups all showing similar positive correlation. 
https://public.tableau.com/profile/todd.stoltey#!/vizhome/Moz_16149003853700/PMP_Sess_Scatter


In [39]:
#11) Extract Moz dataset to csv for upload into Tableau
df_test.to_csv('moz_data.csv', encoding='utf-8')